# Imports

In [1]:
from botiverse import chat_gui
from botiverse.bots import TaskBot

OSError: [WinError 127] The specified procedure could not be found

# Constants

In [ ]:
# The domains e.g. restaurant, hotel, etc.
domains = ["restaurant"]

# The names of the slots where each slot is prefixed with the domain name e.g. restaurant-name, hotel-name, etc.
slot_list = [
    "restaurant-category",
    "restaurant-rating",
    "restaurant-num_people",
    "restaurant-location",
    "restaurant-restaurant_name",
    "restaurant-time",
    "restaurant-date",
    "restaurant-price_range",
    "restaurant-meal"
]

# The utterances for the system to start the conversation
start = [
    {
        'utterance': 'Hi I am Tody, I can help you reserve a restaurant?',
        'slots': [],
        'system_act': {}
    }
]

# The templates for generating responses
templates = [
    {
        'utterance': 'what type of food do you want and in what area?',
        'slots': ['restaurant-location', 'restaurant-category'],
        'system_act': {}
    },
    {
        'utterance': 'what is your preferred price range and rating?',
        'slots': ['restaurant-price_range', 'restaurant-rating'],
        'system_act': {}
    },
    {
        'utterance': 'how many people will be in your party?',
        'slots': ['restaurant-num_people'],
        'system_act': {}
    },
    {
        'utterance': 'what time and date would you like to reserve a table for?',
        'slots': ['restaurant-time', 'restaurant-date'],
        'system_act': {}
    },
    {
        'utterance': 'May I suggest kfc restaurant?',
        'slots': ['restaurant-restaurant_name'],
        'system_act': {'restaurant-restaurant_name': ['kfc']}
    },
    {
        'utterance': 'ok done, here is your reservation number: 123456',
        'slots': [],
        'system_act': {}
    }
]

# Create chatbot

In [ ]:
chatbot = TaskBot(domains, slot_list, start, templates, verbose=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Load pre-trained model on sim-R dataset from botiverse

In [ ]:
chatbot.load_dst_model(pretrained='sim-R')

Model already exists. Skipping download.
Model loaded successfully.


# Run chatbot using GUI from botiverse


In [ ]:
chat_gui('Task bot', chatbot.infer)

 * Serving Flask app 'botiverse.gui.gui'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [25/Jul/2023 00:26:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Jul/2023 00:26:03] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2023 00:26:03] "GET /static/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2023 00:26:03] "GET /static/icons/Task%20bot.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Jul/2023 00:26:06] "POST /get HTTP/1.1" 200 -


hiiiiiiiiiiiiiiiiiiii
State: {}
hiiiiiiiiiiiiiiiiiiii


127.0.0.1 - - [25/Jul/2023 00:26:29] "POST /get HTTP/1.1" 200 -


State: {'restaurant-meal': 'lunch'}


AttributeError: 'chat_gui' object has no attribute 'run'

In [ ]:
print(chatbot.get_dialogue_state())

{}
